In [1]:
import pandas as pd
import folium

# load csv file in dataframe
df = pd.read_csv('data/weather/postesSynop.csv', sep=";")

# filter to keep only station on metropole
df_metro = df[(df['Latitude'] >= 41) & (df['Latitude'] <= 51) & (df['Longitude'] >= -5) & (df['Longitude'] <= 10)]

# create map of France center on middle of France
map_of_france = folium.Map(location=[46.603354, 1.888334], zoom_start=6)  # Ce sont les coordonnées approximatives du centre de la France

# add station to map
for _, row in df_metro.iterrows():
    lat, lon = row['Latitude'], row['Longitude']  # Remplacez 'latitude' et 'longitude' par les noms exacts des colonnes si elles sont différentes
    folium.Marker([lat, lon], tooltip=row['Nom']).add_to(map_of_france)  # Remplacez 'nom_de_la_station' par le nom exact de la colonne contenant le nom de la station si elle est différente

# plot map
map_of_france.save('stations_meteo_france.html')  # Sauvegarde la carte dans un fichier HTML
map_of_france


In [2]:
import pandas as pd



# files list to read
files = [f"data/weather/data/synop.{year}{month:02d}.csv" for year in range(2017, 2024) for month in range(1, 13)]

# list to stock temporary dataframe
dfs = []

for file in files:
    try:
        df = pd.read_csv(file, sep=";")

        # filter on selected column
        cols_of_interest = ['numer_sta', 'date', 't', 'u', 'pres', 'etat_sol', 'rr3']
        df = df[cols_of_interest]

        #transform numerical value in float or Nan if no value
        for col in ['t', 'u', 'pres', 'rr3']:
            df[col] = pd.to_numeric(df[col], errors = 'coerce')

        #convert temperature from Kelvin to Celsius
        df['t'] = df['t'] - 273.15

        # filter based on station existing in df_metro
        df = df[df['numer_sta'].isin(df_metro['ID'])]

        # data column transformation
        df['date'] = df['date'].astype(str).str[:8]  # keep only yyyymmdd 

        # group by dates and make statistics
        grouped = df.groupby(['numer_sta', 'date'])
        daily = grouped.agg({
            't': ['mean', 'min', 'max'],
            'u': 'mean',
            'pres': 'mean',
            'etat_sol': 'max',
            'rr3': 'sum'
        }).reset_index()



        # renamed column
        daily.columns = ['numer_sta', 'date', 't_mean', 't_min', 't_max', 'u_mean', 'pres_mean', 'etat_sol_max', 'rr3_sum']
        #round temperature to 1
        daily['t_mean'] = daily['t_mean'].round(1)
        dfs.append(daily)
        
    except FileNotFoundError:
        # if files doesn't exist, continue
        continue

# concatenante all dataframe in one
df_weather = pd.concat(dfs, ignore_index=True)




In [3]:
import pandas as pd


# replace station id by name
df_weather = df_weather.merge(df_metro[['ID', 'Nom']], left_on='numer_sta', right_on='ID', how='left')
df_weather.drop(columns=['numer_sta', 'ID'], inplace=True)
df_weather.rename(columns={'Name': 'station'}, inplace=True)

# convert date type in datetime
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y%m%d')

# define data as index
df_weather.set_index('date', inplace=True)


print(df_weather)



            t_mean  t_min  t_max     u_mean      pres_mean etat_sol_max  \
date                                                                      
2017-01-01    -1.9   -5.1    0.6  96.875000  101242.500000           mq   
2017-01-02     1.6    0.0    4.4  94.625000  101682.500000           mq   
2017-01-03     2.1   -1.9    6.0  88.500000  101971.250000           mq   
2017-01-04     4.9    0.3    6.8  84.375000  101523.750000           mq   
2017-01-05     2.1    0.2    5.2  91.375000  102366.250000           mq   
...            ...    ...    ...        ...            ...          ...   
2023-08-20    28.3   21.8   34.0  51.500000  101581.250000           mq   
2023-08-21    28.8   22.5   34.9  52.125000  101682.500000           mq   
2023-08-22    29.5   24.6   34.1  47.125000  101487.500000           mq   
2023-08-23    28.7   23.4   33.3  55.500000  101345.000000           mq   
2023-08-24    23.8   22.6   25.3  56.333333  101383.333333           mq   

            rr3_sum     

In [5]:
#complete missing values for etat_sol
# 1. Créez une copie de la table originale
weather_data_copy = df_weather.copy()

# 2. Transformations
weather_data_copy.replace('mq', np.nan, inplace=True)

# 3. Appliquer IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Sélectionnez uniquement les colonnes à imputer
cols_to_impute = ['t_mean', 't_max', 't_min', 'u_mean', 'pres_mean', 'etat_sol_max', 'rr3_sum']
subset_data = weather_data_copy[cols_to_impute]

# Créer l'imputer et appliquer
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = imputer.fit_transform(subset_data)

# Remplacez les colonnes originales par les données imputées
weather_data_copy[cols_to_impute] = imputed_data

# 4. Maintenant, weather_data_copy est votre DataFrame avec des valeurs imputées, que vous pouvez utiliser pour les analyses/modèles suivants.
weather_data_copy['etat_sol_max'] = weather_data_copy['etat_sol_max'].round().clip(lower=0)
weather_data_copy['u_mean'] = weather_data_copy['u_mean'].round(3)
weather_data_copy['pres_mean'] = weather_data_copy['pres_mean'].round(3)

In [6]:
#pivot table
features = ['t_mean', 't_min', 't_max', 'u_mean', 'pres_mean', 'etat_sol_max', 'rr3_sum']

# Pivoter le tableau pour chaque feature
for feature in features:
    pivoted = weather_data_copy.pivot_table(index='date', columns='Nom', values=feature)
    pivoted.columns = [f"{col}_{feature}" for col in pivoted.columns]

    # Si c'est la première feature, initialisez df_pivoted avec cette feature
    if feature == features[0]:
        df_pivoted = pivoted
    # Sinon, joignez les données à df_pivoted
    else:
        df_pivoted = df_pivoted.join(pivoted)

# Le dataframe df_pivoted est maintenant la table remodelée
print(df_pivoted)

            ABBEVILLE_t_mean  AJACCIO_t_mean  ALENCON_t_mean  \
date                                                           
2017-01-01              -1.9             9.3            -3.0   
2017-01-02               1.6            10.5             0.6   
2017-01-03               2.1            10.1            -3.0   
2017-01-04               4.9             8.3             0.9   
2017-01-05               2.1             6.9             0.4   
...                      ...             ...             ...   
2023-08-20              19.4            25.3            20.9   
2023-08-21              17.7            26.4            21.6   
2023-08-22              18.1            26.2            24.2   
2023-08-23              20.9            26.0            22.7   
2023-08-24              21.0            21.9            21.6   

            BALE-MULHOUSE_t_mean  BASTIA_t_mean  BELLE ILE-LE TALUT_t_mean  \
date                                                                         
2017-01-01 

In [7]:
# save dataframe
df_pivoted.to_csv("data/weather/weather_data.csv")

In [7]:
#create dataframe with mean and standard deviation of all station per day

# Convert all columns to float (excluding 'name' column)
for col in weather_data_copy.columns:
    if col != 'Nom':
        weather_data_copy[col] = pd.to_numeric(weather_data_copy[col], errors='coerce')

# Exclude the 'name' column and then compute the mean
df_mean = weather_data_copy.drop(columns=['Nom']).groupby(weather_data_copy.index).mean()

# Exclude the 'name' column and then compute the standard deviation
df_std = weather_data_copy.drop(columns=['Nom']).groupby(weather_data_copy.index).std()




# Renommez les colonnes pour indiquer qu'il s'agit de moyennes et d'écart-types
df_mean.columns = [col + '_mean' for col in df_mean.columns]
df_std.columns = [col + '_std' for col in df_std.columns]

# Fusionnez les deux dataframes le long de l'axe des colonnes
result = pd.concat([df_mean, df_std], axis=1)
print(result)

            t_mean_mean  t_min_mean  t_max_mean  u_mean_mean  pres_mean_mean  \
date                                                                           
2017-01-01     1.904762   -0.783333    5.326190    88.386905   100219.434524   
2017-01-02     2.921429    0.985714    5.461905    89.604167   100204.404762   
2017-01-03     1.821429   -0.385714    4.623810    83.571429   100524.285714   
2017-01-04     2.271429   -1.100000    6.164286    81.056548   100344.404762   
2017-01-05     2.950000    0.321429    5.935714    76.300595   100672.500000   
...                 ...         ...         ...          ...             ...   
2023-08-20    24.575738   18.876730   30.571145    65.823286    99995.752952   
2023-08-21    25.013833   19.424349   30.966383    64.571143   100073.537810   
2023-08-22    25.349547   19.888634   31.137812    64.398857    99882.147500   
2023-08-23    25.944785   19.879111   32.685431    63.155738    99669.834571   
2023-08-24    22.544785   21.593396   23

In [9]:

result.to_csv("data/weather/weather_stats.csv")